In [2]:
import torch

In [3]:
import numpy

In [5]:
test = torch.tensor([0,2,3])

In [6]:
test

tensor([0, 2, 3])

In [5]:
torch.cuda.is_available()

True

In [7]:
device = torch.device('cuda')

In [8]:
device

device(type='cuda')